In [9]:
pip install tensorflow pandas matplotlib sklearn numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import os
import pandas as pd
import tensorflow as tf


In [11]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')

In [12]:
df = df.head(100000)

In [13]:
df.iloc[0].comment_text

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [14]:
from tensorflow.keras.layers import TextVectorization

In [15]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [16]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [17]:
max_feats = 150000

In [18]:
vectorizer = TextVectorization(max_tokens = max_feats,
                              output_sequence_length = 1800,
                              output_mode = 'int')

In [19]:
vectorizer.adapt(X.values)

In [20]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'are',
 'have',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'wikipedia',
 'so',
 'can',
 'what',
 'there',
 'has',
 'all',
 'will',
 'talk',
 'please',
 'no',
 'its',
 'would',
 'one',
 'like',
 'just',
 'they',
 'he',
 'dont',
 'which',
 'been',
 'any',
 'should',
 'more',
 'we',
 'other',
 'some',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'edit',
 'why',
 'articles',
 'people',
 'only',
 'out',
 'up',
 'were',
 'when',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'their',
 'being',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'information',
 'very',
 'does',
 'well',
 'could',
 'such',
 'want',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'first',
 'pages',
 'new',

In [21]:
vectorizer("Hello how are you")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([291,  73,  19, ...,   0,   0,   0])>

In [22]:
vectorized_text = vectorizer(X.values)

In [23]:
vectorized_text

<tf.Tensor: shape=(100000, 1800), dtype=int64, numpy=
array([[  652,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2519, ...,     0,     0,     0],
       [  410,   432,    70, ...,     0,     0,     0],
       ...,
       [   12,   290,    11, ...,     0,     0,     0],
       [24350,    23,     7, ...,     0,     0,     0],
       [  171,   456,   326, ...,     0,     0,     0]])>

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(90000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [25]:
batch_X,batch_y = dataset.as_numpy_iterator().next()

In [26]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional,Dense,Dropout,Embedding

In [28]:
model = Sequential()
model.add(Embedding(max_feats+1,32))
model.add(Bidirectional(LSTM(32,activation = 'tanh')))
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(6,activation = 'sigmoid'))

In [29]:
model.compile(loss='BinaryCrossentropy',optimizer='adam',metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          4800032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [31]:
history = model.fit(train,epochs = 5,validation_data = val)

Epoch 1/5
4375/4375 [==============================] - 542s 122ms/step - loss: 0.0686 - accuracy: 0.9716 - val_loss: 0.0462 - val_accuracy: 0.9937
Epoch 2/5
4375/4375 [==============================] - 447s 102ms/step - loss: 0.0471 - accuracy: 0.9832 - val_loss: 0.0403 - val_accuracy: 0.9947
Epoch 3/5
4375/4375 [==============================] - 395s 90ms/step - loss: 0.0436 - accuracy: 0.9866 - val_loss: 0.0374 - val_accuracy: 0.9945
Epoch 4/5
4375/4375 [==============================] - 397s 91ms/step - loss: 0.0382 - accuracy: 0.9770 - val_loss: 0.0353 - val_accuracy: 0.9936
Epoch 5/5
4375/4375 [==============================] - 392s 90ms/step - loss: 0.0352 - accuracy: 0.9841 - val_loss: 0.0314 - val_accuracy: 0.9938


In [32]:
input_text = vectorizer("You freaking suck!")

In [33]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [35]:
import numpy as np

In [36]:
res = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 1s 685ms/step


In [37]:
(res > 0.5).astype(int)

array([[1, 0, 1, 0, 1, 0]])

In [38]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [39]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 72ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [40]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [45]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy
import re

In [46]:
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [47]:
for batch in test.as_numpy_iterator():
    X_true,y_true = batch
    yhat = model.predict(X_true)
    
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true,yhat)
    rec.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)

1/1 [==============================] - 0s 69ms/step


In [49]:
print(f'Precision: {pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8484569191932678, Recall:0.8278651833534241, Accuracy:0.4943999946117401


In [50]:
pip install gradio jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 

In [51]:
import gradio as gr

In [52]:
model.save('toxicity.h5')

In [53]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [54]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [55]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://371ba5f230d1e168c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
